In [1]:
%reset -f
import h5py
import time as t
import numpy as np
import scipy as sp
import scipy.io as spi
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_probability as tfp

data_train = spi.loadmat('train_PDE_DR.mat')

u_in = data_train['X_train0'][0:50000,:]
x_t_in = data_train['X_train1'][0:50000,:]
s_in = data_train['y_train'][0:50000,:]

u_in.shape, x_t_in.shape, s_in.shape

((50000, 100), (50000, 2), (50000, 1))

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Lambda, Dense

bs = 50000

def fn(x):
    y = tf.einsum("ij, ij->i", x[0], x[1])
    y = tf.expand_dims(y, axis = 1)
    return y

tfd = tfp.distributions
tfb = tfp.bijectors

def normal_sp(params):
    return tfd.Normal(loc = params[:, 0:1], scale = 0.001+tf.math.softplus(params[:, 1:2]))    

def negloglikelihood(y_true, y_pred):
    return tf.keras.backend.sum(-y_pred.log_prob(y_true))+(sum(model.losses)/bs)

hln = 25

inputsB = Input(shape = (100,), name = 'inputsB')
hiddenB = tfp.layers.DenseFlipout(hln, activation = "relu")(inputsB)
hiddenB = tfp.layers.DenseFlipout(hln, activation = "relu")(hiddenB)
hiddenB = tfp.layers.DenseFlipout(hln, activation = "relu")(hiddenB)
hiddenB = tfp.layers.DenseFlipout(hln, activation = "relu")(hiddenB)

inputsT = Input(shape = (2,), name = 'inputsT')
hiddenT = tfp.layers.DenseFlipout(hln, activation = "relu")(inputsT)
hiddenT = tfp.layers.DenseFlipout(hln, activation = "relu")(hiddenT)
hiddenT = tfp.layers.DenseFlipout(hln, activation = "relu")(hiddenT)
hiddenT = tfp.layers.DenseFlipout(hln, activation = "relu")(hiddenT)

combined = Lambda(fn, output_shape = [None, 1])([hiddenB, hiddenT])
output = tfp.layers.DenseFlipout(2)(combined)

dist = tfp.layers.DistributionLambda(normal_sp)(output)
model = Model(inputs = [inputsB, inputsT], outputs = dist)    

model.summary()

/home/user/anaconda3/envs/SG_Env_TFP/lib/python3.9/site-packages/tensorflow_probability/python/layers/util.py:99: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  loc = add_variable_fn(
2022-10-24 13:30:26.157439: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-10-24 13:30:26.157527: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-10-24 13:30:26.159473: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is opt

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputsB (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 inputsT (InputLayer)           [(None, 2)]          0           []                               
                                                                                                  
 dense_flipout (DenseFlipout)   (None, 25)           5025        ['inputsB[0][0]']                
                                                                                                  
 dense_flipout_4 (DenseFlipout)  (None, 25)          125         ['inputsT[0][0]']                
                                                                                              

2022-10-24 13:30:26.993811: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [3]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.005)
spe = 25
string = './model/TRIALmodel1000C3_S1'

@tf.function
def train_step():
    with tf.GradientTape() as tape:
        loss_value = 0
        for i in range(0,spe):        
            logits = model({"inputsB":u_in, "inputsT":x_t_in}, training=True)
            loss_value = loss_value + negloglikelihood(s_in, logits)
        loss_value = loss_value*(1/spe)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return loss_value

epochs = 10000
loss = np.zeros(epochs)

for epoch in range(epochs):
    loss_value = train_step()
    loss[epoch] = loss_value.numpy()
    if loss[epoch] <= np.min(loss[0:epoch+1]):
        model.save_weights(string)
        last_saved_wt = epoch
    if epoch%10 == 0:
        print("Epoch %d, loss %.2f" % (epoch, loss[epoch]))

print(last_saved_wt)

Epoch 0, loss 39877.29
Epoch 10, loss 38531.90
Epoch 20, loss 36072.61
Epoch 30, loss 33013.02
Epoch 40, loss 30713.41
Epoch 50, loss 28726.76
Epoch 60, loss 26616.61
Epoch 70, loss 25168.88
Epoch 80, loss 23819.35
Epoch 90, loss 22713.27
Epoch 100, loss 21371.43
Epoch 110, loss 19408.15
Epoch 120, loss 17520.28
Epoch 130, loss 16635.89
Epoch 140, loss 13187.33
Epoch 150, loss 10226.96
Epoch 160, loss 7357.02
Epoch 170, loss 5567.42
Epoch 180, loss 3769.03
Epoch 190, loss -1537.37
Epoch 200, loss -5026.68
Epoch 210, loss -7830.66
Epoch 220, loss -10840.90
Epoch 230, loss -9039.54
Epoch 240, loss -14135.99
Epoch 250, loss -13071.82
Epoch 260, loss -14424.20
Epoch 270, loss -20474.61
Epoch 280, loss -19320.96
Epoch 290, loss -6328.44
Epoch 300, loss -18255.17
Epoch 310, loss -23530.44
Epoch 320, loss -29075.37
Epoch 330, loss -31208.97
Epoch 340, loss -32969.49
Epoch 350, loss -34637.25
Epoch 360, loss -34674.25
Epoch 370, loss -29429.44
Epoch 380, loss -17477.26
Epoch 390, loss -21171.8

In [5]:
data_test = spi.loadmat('test_PDE_DR.mat')

u_in_test = data_test['X_test0']
x_t_in_test = data_test['X_test1']
s_in_test = data_test['y_test']

nsamples = 100
pred = np.zeros([nsamples,1000000])
for i in range(0,nsamples):
    if i%5 == 0:
        print(i)
    pred[i,:] = np.squeeze((model({"inputsB":u_in_test, "inputsT":x_t_in_test})).sample(1))
    
print()
print(np.mean((s_in_test-np.mean(pred, axis = 0)[..., np.newaxis])**2))
print(np.mean((s_in_test)**2))
print(np.mean((s_in_test-np.mean(pred, axis = 0)[..., np.newaxis])**2)/np.mean((s_in_test)**2))    

0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95

0.0016519009111849896
0.23837758612778512
0.006929766082535414


In [6]:
model.save_weights('./model/TRIALmodel1000C3_S1')